In [1]:
import pandas as pd

In [2]:
import numpy as np

import typing
from typing import Any, Tuple

import tensorflow as tf

import tensorflow_text as tf_text
import keras
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [98]:
import jax
from jax import numpy as jnp
from flax import linen as nn
import flax
from jax import vmap

In [4]:
df = pd.read_csv('dataset/train.csv')
df.head()

,Fon,French
0,Mi nɔ xwe do Mawu sɛ lɛ,Servez Dieu de toute votre âme.
1,zɔ̀n afɔ̀ yì tomὲ,voyager à pied
2,A na bló alɔkpa ɖokpó ɔ́,Tu feras de la même manière.
3,ɖo azɔxɔ mɛ ɔ mi nɔ mɔ akluzu tabilo tavo wɛma...,en classe nous voyons un crucifix des tableaux...
4,A ta zogbεn ɔ,Tu as allumé la lampe


In [5]:
pairs = []
for i in range(len(df)):
    pairs.append([df['Fon'][i].lower(), df['French'][i].lower()])

In [6]:
inp = df['Fon']
targ = df['French']

In [7]:
BUFFER_SIZE = len(inp)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((inp, targ)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [8]:
for example_input_batch, example_target_batch in dataset.take(1):
    print(example_input_batch[:5])
    print()
    print(example_target_batch[:5])
    break

tf.Tensor(
[b'Ayijay\xc3\xac s\xc9\x94\xcc\x81 t\xc3\xadin \xc9\x96\xc3\xb2 t\xc3\xb2 \xc9\x94\xcc\x81 m\xc9\x9b\xcc\x80 \xc4\x83'
 b'asi t\xc9\x94n \xc9\x96o fi \xc4\x83 e \xc9\x96o d\xc9\x94n'
 b'un s\xc3\xa8 x\xc3\xb3 e a \xc9\x96\xc9\x94 l\xc9\x9b\xcc\x81'
 b'mi \xc9\x96o jij\xc9\x94 bo n\xc9\x94 \xc9\x96e nu xixa mit\xc9\x94n l\xce\xb5'
 b'mi yi wan nu m\xc9\x9bjit\xc9\x94 mit\xc9\x94n l\xc9\x9b'], shape=(5,), dtype=string)

tf.Tensor(
[b'Il n\xe2\x80\x99y a plus de paix dans le pays.'
 b"sa femme n'est pas ici elle est l\xc3\xa0"
 b"j'ai entendu les paroles que tu as dites"
 b"nous avons l'habitude d'illustrer nos r\xc3\xa9citations"
 b'aimez vos parents'], shape=(5,), dtype=string)


In [9]:
def tf_lower_and_split_punct(text):
    # Split accecented characters.
    text = tf_text.normalize_utf8(text, 'NFKD')
    text = tf.strings.lower(text)
    # Add spaces around punctuation.
    text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
    # Strip whitespace.
    text = tf.strings.strip(text)

    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text

In [10]:
example_text = tf.constant(inp[0])

In [11]:
print(example_text.numpy().decode())
print(tf_lower_and_split_punct(example_text).numpy().decode())

Mi nɔ xwe do Mawu sɛ lɛ
[START] mi nɔ xwe do mawu sɛ lɛ [END]


In [12]:


input_text_processor = keras.layers.TextVectorization(
        standardize=tf_lower_and_split_punct,
      )

In [13]:
input_text_processor.adapt(inp)
input_text_processor.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('[START]'),
 np.str_('[END]'),
 np.str_('e'),
 np.str_('ɖo'),
 np.str_('ɔ'),
 np.str_('a'),
 np.str_('un'),
 np.str_('-')]

In [14]:
len(input_text_processor.get_vocabulary())

13444

In [15]:
output_text_processor = keras.layers.TextVectorization(
        standardize=tf_lower_and_split_punct)

output_text_processor.adapt(targ)
output_text_processor.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('[START]'),
 np.str_('[END]'),
 np.str_('.'),
 np.str_('il'),
 np.str_('de'),
 np.str_('la'),
 np.str_('le'),
 np.str_('est')]

In [16]:
dataset = tf.data.Dataset.from_tensor_slices((inp, targ)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [17]:
for example_input_batch, example_target_batch in dataset.take(1):
    print(example_input_batch[:5])
    print()
    print(example_target_batch[:5])
    break

tf.Tensor(
[b'xo w\xc9\x9b \xc3\xa9 \xc9\x96\xc3\xa8'
 b'xwe wo \xc9\x96ie \xc9\x94 yewunk\xc9\x94nnyi\xc9\x96owuxwe \xc9\x96okpo do ali el\xc9\x94 ji'
 b'hwl\xce\xb5\xcc\x81n k\xc3\xa0n x\xc3\xa1 m\xce\xb5'
 b'j\xc3\xb2n\xc9\x94\xcc\x80 b\xc3\xa1 m\xc3\xac'
 b"akw\xc9\x9b\xcc\x81 ma \xc9\x96'\xc3\xa0s\xc3\xad w\xc3\xa8 h\xc5\xadn gb\xc9\x94j\xc9\x9b\xcc\x81"], shape=(5,), dtype=string)

tf.Tensor(
[b'elle est enceinte'
 b'il y a dix ans il y avait un cin\xc3\xa9ma dans cette rue'
 b'forcer une femme \xc3\xa0 commettre un adult\xc3\xa8re'
 b"j'ai un visiteur" b"si tu n'as pas d'argent reste tranquille"], shape=(5,), dtype=string)


In [18]:
embedding_dim = 256
units = 1024

In [19]:

class ShapeChecker:
    def __call__(self, tensor, expected_shape):
        assert tensor.shape == expected_shape, f"Expected {expected_shape}, got {tensor.shape}"



In [46]:
class Encoder(nn.Module):
    input_vocab_size: int
    embedding_dim: int
    enc_units: int

    @nn.compact
    def __call__(self, tokens,):

        shape_checker = ShapeChecker()

        embedding = nn.Embed(num_embeddings=self.input_vocab_size, features=self.embedding_dim)
        gru = nn.RNN(nn.GRUCell(self.enc_units),return_carry=True)
        batch_size, seq_length = tokens.shape
        shape_checker(tokens, (batch_size, seq_length))

        vectors = embedding(tokens)
        shape_checker(vectors, (batch_size, seq_length, self.embedding_dim))
        _, enc_output = gru(vectors, seq_lengths=seq_length)
        shape_checker(enc_output, (batch_size, seq_length, self.enc_units))
        return enc_output

In [47]:
encoder = Encoder(input_text_processor.vocabulary_size(),
                  embedding_dim, units)

In [48]:
example_tokens = input_text_processor(example_input_batch)

In [49]:
params = encoder.init(jax.random.PRNGKey(0), jnp.array(example_tokens))

In [50]:
example_tokens

<tf.Tensor: shape=(64, 14), dtype=int64, numpy=
array([[    2,    63,    24,    13,    72,     3,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    2,    52,   390,   235,     6,  3106,    54,    42,   119,
          152,    50,     3,     0,     0],
       [    2,  7176,   239,   158,    20,     3,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    2, 10979,   536,    30,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    2,   342,    45,  2024,    90,   279,   955,     3,     0,
            0,     0,     0,     0,     0],
       [    2,  5302,  2823,     9,  4248,     9,  2946,     9,  3433,
            3,     0,     0,     0,     0],
       [    2,   225,  2875,     3,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    2,     4,    73,   219,    17,   153,    13,  1122,  8804,
            3,     0,     0,     0,     0],
       [    2,     7,    22,   1

In [51]:
jnp.array(example_tokens)

Array([[    2,    63,    24,    13,    72,     3,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    2,    52,   390,   235,     6,  3106,    54,    42,   119,
          152,    50,     3,     0,     0],
       [    2,  7176,   239,   158,    20,     3,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    2, 10979,   536,    30,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    2,   342,    45,  2024,    90,   279,   955,     3,     0,
            0,     0,     0,     0,     0],
       [    2,  5302,  2823,     9,  4248,     9,  2946,     9,  3433,
            3,     0,     0,     0,     0],
       [    2,   225,  2875,     3,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    2,     4,    73,   219,    17,   153,    13,  1122,  8804,
            3,     0,     0,     0,     0],
       [    2,     7,    22,   135,   157,    34,     3,     0,     0,
         

In [52]:
output= encoder.apply(params, jnp.array(example_tokens))

In [53]:
output.shape

(64, 14, 1024)

In [204]:
class LuongAttention(nn.Module):
    units: jnp.int8  # Nombre d'unités pour la projection

    @nn.compact
    def __call__(self, decoder_output, encoder_output):
        shape_checker = ShapeChecker()

        # Projetez l'encodeur et le décodeur dans le même espace de dimensions
        wa_enc = nn.Dense(features=self.units, use_bias=False)  # Pour encoder_output
        wa_dec = nn.Dense(features=self.units, use_bias=False)  # Pour decoder_output
        va = nn.Dense(features=1, use_bias=False)  # Projection pour score scalaire

        # Agrandissez decoder_output pour qu'il corresponde au temps séquentiel de l'encodeur
        decoder_output = jnp.tile(decoder_output, [1, encoder_output.shape[1], 1])  # (batch_size, seq_len, units)
        shape_checker(decoder_output, (encoder_output.shape[0], encoder_output.shape[1], decoder_output.shape[-1]))

        # Calcul du score (produit scalaire + activation tanh)
        score = va(nn.tanh(wa_enc(encoder_output) + wa_dec(decoder_output)))  # (batch_size, seq_len, 1)
        shape_checker(score, (encoder_output.shape[0], encoder_output.shape[1], 1))

        # Normalisation via softmax sur l'axe temporel (axis=1)
        score = nn.softmax(score, axis=1)  # Toujours (batch_size, seq_len, 1)
        shape_checker(score, (encoder_output.shape[0], encoder_output.shape[1], 1))

        score = jnp.transpose(score, (0, 2, 1))

        alignment = nn.softmax(score, axis=-1)  # (batch_size, 1, seq_len)
        shape_checker(alignment, (encoder_output.shape[0], 1, encoder_output.shape[1]))
        context = jnp.matmul(alignment, encoder_output)  # (batch_size, 1, units)
        shape_checker(context, (encoder_output.shape[0], 1, encoder_output.shape[2]))
        return context, alignment



In [205]:
attention = LuongAttention(units)

In [215]:
example_attention_query = tf.random.normal(shape=[len(example_tokens), 1, 1024])
example_attention_query = jnp.array(example_attention_query)

In [216]:
params_att = attention.init(jax.random.PRNGKey(0), example_attention_query, output)

In [217]:
context, alignment = attention.apply(params_att, example_attention_query, output)

In [218]:
context.shape

(64, 1, 1024)

In [219]:
alignment.shape

(64, 1, 14)

In [220]:
class DecoderInput(typing.NamedTuple):
    new_tokens: Any
    enc_output: Any

class DecoderOutput(typing.NamedTuple):
    logits: Any
    attention_weights: Any

In [316]:
class Decoder(nn.Module):
    output_vocab_size: jnp.int8
    embedding_dim: jnp.int8
    dec_units: jnp.int8

    @nn.compact
    def __call__(self, inputs: DecoderInput,
                 state=None):
        shape_checker = ShapeChecker()

        vectors = nn.Embed(num_embeddings=self.output_vocab_size, features=self.embedding_dim)(inputs.new_tokens)
        #shape_checker(vectors, (inputs.new_tokens.shape[0], 1,  self.embedding_dim))

        gru = nn.RNN(nn.GRUCell(self.dec_units), return_carry=True)
        state, rnn_out = gru(vectors)
    #shape_checker(rnn_out, (inputs.new_tokens.shape[0], 1, self.dec_units))

        context, alignment = LuongAttention(self.dec_units)(rnn_out, inputs.enc_output)
        #shape_checker(context, (inputs.new_tokens.shape[0], 1, self.dec_units))
        #shape_checker(alignment, (inputs.new_tokens.shape[0], 1, inputs.enc_output.shape[1]))

        context_and_rnn_output = jnp.concatenate([context, rnn_out], axis=-1)
        Wc = nn.Dense(features=self.dec_units, use_bias=False)
        attention_vector = jnp.tanh(Wc(context_and_rnn_output))
        #shape_checker(attention_vector, (inputs.new_tokens.shape[0], 1, self.dec_units))

        logits = nn.Dense(features=self.output_vocab_size)(attention_vector)
        #shape_checker(logits, (inputs.new_tokens.shape[0], 1, self.output_vocab_size))
        return DecoderOutput(logits=logits, attention_weights=alignment)

In [317]:
decoder = Decoder(output_text_processor.vocabulary_size(),embedding_dim, units)

In [318]:
example_output_tokens = output_text_processor(example_target_batch)

start_index = output_text_processor.get_vocabulary().index('[START]')
first_token = tf.constant([[start_index]] * example_output_tokens.shape[0])

In [319]:
first_token = jnp.array(first_token)
example_output_tokens = jnp.array(example_output_tokens)

In [320]:
params_dec = decoder.init(jax.random.PRNGKey(0),  DecoderInput(new_tokens=first_token,
                                                               enc_output=output))

In [321]:
decoder_output = decoder.apply(params_dec, DecoderInput(new_tokens=first_token,
                                                        enc_output=output))

In [322]:
class Seq2Seq(nn.Module):
    encoder: Encoder
    decoder: Decoder

    @nn.compact
    def __call__(self, encoder_input_tokens, decoder_input_tokens):
        enc_output = self.encoder(encoder_input_tokens)
        decoder_input = DecoderInput(new_tokens=decoder_input_tokens, enc_output=enc_output)
        decoder_output = self.decoder(decoder_input)
        return decoder_output

In [323]:
seq2seq = Seq2Seq(encoder, decoder)

In [324]:
params_seq2seq = seq2seq.init(jax.random.PRNGKey(0),jnp.array(example_tokens), first_token )

In [325]:
import optax

optimizer = optax.adam(1e-3)

In [326]:
from flax.training import train_state
model_state = train_state.TrainState.create(apply_fn=seq2seq.apply, params=params_seq2seq, tx=optimizer)

In [327]:
def calculate_loss(state, params, batch):
    data_inputs, labels = batch
    data_inputs = jnp.array(data_inputs)
    labels = jnp.array(labels)

    max_target_length = labels.shape[1]
    loss = 0.0
    for i in range(max_target_length-1):
        new_labels = labels[:, i:i+2]
        logits = seq2seq.apply(params, data_inputs, new_labels)
        logits = logits.squeeze(axis=-1)

        step_loss = optax.sigmoid_binary_cross_entropy(logits, new_labels)
        step_loss = loss.mean()
        loss += step_loss
    return loss






In [328]:
grad_fn = jax.value_and_grad(calculate_loss, argnums=1)

In [329]:

def train_step(state, batch):
    loss, grads = grad_fn(state, state.params, batch)
    new_state = state.apply_gradients(grads=grads)
    return new_state, loss


In [330]:
def train_epoch(state, train_ds):
    losses = []
    for batch in train_ds:
        state, loss = train_step(state, batch)
        losses.append(loss)
    return state

In [331]:
input_text_processor

<TextVectorization name=text_vectorization, built=True>

In [332]:
trained_state = train_epoch(model_state, dataset_jax)

AssertionError: Expected (64, 27, 1024), got (64, 54, 1024)

In [281]:
type(dataset)

tensorflow.python.data.ops.batch_op._BatchDataset

In [282]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.string, name=None))>

In [293]:

dataset_jax = []
for a, b in dataset:
    dataset_jax.append((jnp.array(input_text_processor(a)), jnp.array(output_text_processor(b))))
    break

In [294]:
len(dataset)

842

In [296]:
dataset_jax

[(Array([[   2,  197,   25, ...,    0,    0,    0],
         [   2,  190,  123, ...,    0,    0,    0],
         [   2,   58,  303, ...,    0,    0,    0],
         ...,
         [   2,  185,  113, ...,    0,    0,    0],
         [   2, 7575,    3, ...,    0,    0,    0],
         [   2,  735,  329, ...,    0,    0,    0]], dtype=int32),
  Array([[    2,    13,   460, ...,     0,     0,     0],
         [    2,    83,    75, ...,     0,     0,     0],
         [    2,   326,     3, ...,     0,     0,     0],
         ...,
         [    2,    70,   128, ...,     0,     0,     0],
         [    2,    28,   893, ...,     0,     0,     0],
         [    2, 13551,     8, ...,     0,     0,     0]], dtype=int32))]